In [105]:
import pandas
import numpy
df_raw = pandas.read_json("raw_data.json")

In [106]:
# Variables

reforges = [
    "\u278c",
    "\u25c6",
    "\u278a",
    "\u278e",
    "\u278b",
    "\u273f",
    "\u278d",
    " ✦",
    "⚚ ",
    " ✪",
    "✪",
    "Stiff ",
    "Lucky ",
    "Jerry's ",
    "Dirty ",
    "Fabled ",
    "Suspicious ",
    "Gilded ",
    "Warped ",
    "Withered ",
    "Bulky ",
    "Stellar ",
    "Heated ",
    "Ambered ",
    "Fruitful ",
    "Magnetic ",
    "Fleet ",
    "Mithraic ",
    "Auspicious ",
    "Refined ",
    "Headstrong ",
    "Precise ",
    "Spiritual ",
    "Moil ",
    "Blessed ",
    "Toil ",
    "Bountiful ",
    "Candied ",
    "Submerged ",
    "Reinforced ",
    "Cubic ",
    "Warped ",
    "Undead ",
    "Ridiculous ",
    "Necrotic ",
    "Spiked ",
    "Jaded ",
    "Loving ",
    "Perfect ",
    "Renowned ",
    "Giant ",
    "Empowered ",
    "Ancient ",
    "Sweet ",
    "Silky ",
    "Bloody ",
    "Shaded ",
    "Gentle ",
    "Odd ",
    "Fast ",
    "Fair ",
    "Epic ",
    "Sharp ",
    "Heroic ",
    "Spicy ",
    "Legendary ",
    "Deadly ",
    "Fine ",
    "Grand ",
    "Hasty ",
    "Neat ",
    "Rapid ",
    "Unreal ",
    "Awkward ",
    "Rich ",
    "Clean ",
    "Fierce ",
    "Heavy ",
    "Light ",
    "Mythic ",
    "Pure ",
    "Smart ",
    "Titanic ",
    "Wise ",
    "Bizarre ",
    "Itchy ",
    "Ominous ",
    "Pleasant ",
    "Pretty ",
    "Shiny ",
    "Simple ",
    "Strange ",
    "Vivid ",
    "Godly ",
    "Demonic ",
    "Forceful ",
    "Hurtful ",
    "Keen ",
    "Strong ",
    "Superior ",
    "Unpleasant ",
    "Zealous ",
]

df = df_raw.query("bin == False")
df = df[["uuid","item_name","starting_bid","tier"]][~df["item_lore"].str.contains(r"\bfurniture\b")]
df["item_id"] = df["item_name"].str.replace('|'.join(reforges), '', regex=True).replace(r"\[[^\]]*\] ", "", regex=True).replace(r'^\s+', '', regex=True)
df["item_id"] = df['item_id'].astype(str) + '|' + df['tier'].astype(str)
df = df.groupby('item_id').agg({'uuid': lambda x: list(x), 'item_name': lambda x: list(x), 'starting_bid': lambda x: list(x), 'tier': lambda x: list(x)})
df = df[df['tier'].str.len() >= 2]


In [107]:
items = pandas.DataFrame(columns=["item_name", "uuid", "price", "tier", "profit", "profit_percent", "mean", "median", "std", "rating"])
items_index = 0


for index, row in df.iterrows():

    prices = row["starting_bid"]
    modal_index = prices.index(min(prices))
    prices = sorted(prices)

    price = prices[0]
    profit = prices[1] - price
    profit_percent = (profit / price) * 100

    if profit < 500: continue

    uuid = row["uuid"][modal_index]
    tier = row["tier"][modal_index]
    item_name = row["item_name"][modal_index]

    count = len(prices)
    mean = numpy.mean(prices).astype(int)
    std = numpy.std(prices).astype(int)
    median = numpy.median(prices).astype(int)
    
    rating = count * 5
    rating += profit_percent*10
    rating += (1 / ( ( std / mean ) * 100 )) *10000
    rating += ((median - mean) / mean) *100
    rating = int(rating)

    items.loc[items_index] = {
        "uuid": uuid,
        "item_name": item_name,
        "price": price,
        "tier": tier,
        "profit": profit,
        "profit_percent": profit_percent,
        "mean": mean,
        "median": median,
        "std": std,
        "rating": rating
    }

    items_index += 1

In [111]:
sorted_df = items.sort_values("rating", ascending=False)
rating_and_item_name = sorted_df[["rating", "item_name"]]
print(rating_and_item_name)

         rating                                   item_name
82   1599999110                           Gemstone Gauntlet
217  1499999196  ⚚ Ancient Shadow Assassin Chestplate ✪✪✪✪✪
18   1304346936                          [Lvl 63] Black Cat
78   1299999110                Fabled Flower of Truth ✪✪✪✪✪
202   371013602                           Reaper Scythe ✪✪✪
..          ...                                         ...
40          425                          Pure Crimson Boots
159         365                             Molten Bracelet
154         292                              Miner Leggings
273         236                      ⚚ Unreal Venom's Touch
100         201                        Golem Armor Leggings

[297 rows x 2 columns]


In [108]:
print(len(df_raw))
print(len(df))
print(len(items))
items.to_json("pandas_final.json", indent=4)

57346
738
297
